In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd 
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC


In [ ]:
%matplotlib inline
sns.set(color_codes=True)
pal = sns.color_palette("Set2", 10)
sns.set_palette(pal)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
train.info()

In [ ]:
train.head()

## Sex-Survived Comparison

In [ ]:
sns.countplot(x='Sex',hue='Survived',data=train)

In [ ]:
train.isnull().sum()

## Handling Missing values for
1. Age
2. Ticket
3. Fare
4. Embarked
5. Cabin

In [ ]:
##Age
sns.displot(train, x="Age", kind="kde",hue='Survived')

In [ ]:
plt.hist(x=train.Age, bins=10)
plt.grid(axis='y', alpha=0.5)
plt.xlabel('Age')

In [ ]:
print('For Missing Values')
print('Total no. of observations:', train[train['Age'].isnull()].shape[0])
a = train[train['Age'].isnull()].groupby('Survived').Survived.count()
b =  pd.DataFrame({'Survived': a.index, '%age':a.values*100/a.values.sum()})
print(b.to_string(index=False))

In [ ]:
mu=int(train['Age'].mean())
print('For mean')
print('Total no. of observations:', train[train['Age']==mu].shape[0])
a = train[train['Age']==mu].groupby('Survived').Survived.count()
b =  pd.DataFrame({'Survived': a.index, '%age':a.values*100/a.values.sum()})
print(b.to_string(index=False))

### Distribution matches with mean

In [ ]:
train['Age'].fillna(train['Age'].mean(),inplace=True)
test['Age'].fillna(train['Age'].mean(),inplace=True)

In [ ]:
##Ticket
sns.displot(train, x="Fare", kind="kde",hue='Survived')

### A very imbalanced distribution in which we can see high fare passengers survived more than others

In [ ]:
train.groupby('Pclass').Fare.mean()

### As we can see Fare is corelated to the Pclass so we can impute accordingly

In [ ]:
c1 = 92.65
c2 = 25.76
c3 = 20.62
l=[]
s=set(train['Fare'])
for i in range(train['Fare'].shape[0]):
    if train['Fare'][i] not in s:
        if train['Pclass'][i]==1:
            l.append(c1)
        elif train['Pclass'][i]==2:
            l.append(c2)
        else:
            l.append(c3)
    else:
        l.append(train['Fare'][i])
        
train['Fare'] = l

l=[]
s=set(test['Fare'])
for i in range(test['Fare'].shape[0]):
    if test['Fare'][i] not in s:
        if test['Pclass'][i]==1:
            l.append(c1)
        elif test['Pclass'][i]==2:
            l.append(c2)
        else:
            l.append(c3)
    else:
        l.append(test['Fare'][i])
        
test['Fare'] = l

In [ ]:
##Ticket
train.groupby('Ticket').Ticket.count().sort_values(ascending=False).head(20)


In [ ]:
train.groupby(['Ticket','Survived']).Survived.count().sort_values(ascending=False).head(20)

In [ ]:
train['Ticket'] = train['Ticket'].str.replace('[^a-zA-Z]', 'r').str[:1]

test['Ticket'] = test['Ticket'].str.replace('[^a-zA-Z]', 'r').str[:1]

In [ ]:
train['Ticket'].fillna('Random',inplace=True)
l=[]
for i in train['Ticket']:
    if i == 'r':
        l.append('Random')
    else:
        l.append(i)
        
train['Ticket'] = l


test['Ticket'].fillna('Random',inplace=True)
l=[]
for i in test['Ticket']:
    if i == 'r':
        l.append('Random')
    else:
        l.append(i)
        
test['Ticket'] = l


In [ ]:
train.groupby('Ticket').Ticket.count().sort_values(ascending=False).head(20)

In [ ]:
sns.countplot(x='Ticket',hue='Survived',data=train)

In [ ]:
#Embarked
train.groupby('Embarked').Embarked.count()

In [ ]:
sns.countplot(x='Embarked',hue='Survived',data=train)

In [ ]:
train.groupby(['Embarked','Survived']).Survived.count()

In [ ]:
train[train['Embarked'].isnull()].groupby('Survived').Survived.count()

### Here we have two options to impute the mode that is **S** or impute the category which has most equal ratio as survived that is **Q** So we can do trial and error

In [ ]:
# imputing Q
l=[]
for i in train['Embarked']:
    if i in[np.nan]:
        l.append('S')
    else:
        l.append(i)

train['Embarked']=l

l=[]
for i in test['Embarked']:
    if i in[np.nan]:
        l.append('S')
    else:
        l.append(i)

test['Embarked']=l

## Cabin

In [ ]:
train['Cabin'].value_counts()

In [ ]:
train['Cabin'] = train['Cabin'].str[0:1]
train['Cabin'].fillna('Random',inplace=True)
test['Cabin'] = test['Cabin'].str[0:1]
test['Cabin'].fillna('Random',inplace=True)

In [ ]:
train['Cabin'].value_counts()

In [ ]:
sns.countplot(x='Cabin',hue='Survived',data=train)

## Every missing value has been handled

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['LastName'] = train['Name'].str.split(', ').str[0:1]
test['LastName'] = test['Name'].str.split(', ').str[0:1]

In [ ]:
l=[]
for i in train['LastName']:
    
    l.append(i[0])
    
train['LastName'] = l


l=[]
for i in test['LastName']:
    l.append(i[0])
    
test['LastName'] = l

In [ ]:
df = train.groupby('LastName').Survived.agg(['count','mean']).sort_values(by='count',ascending=False)
df = df.reset_index()
df.head(20)


In [ ]:
df2 = test.groupby('LastName').LastName.agg(['count']).sort_values(by='count',ascending=False)
df2 = df2.reset_index()
df2.head(20)


In [ ]:
d = {}
for i in range(df.shape[0]):
    d[df['LastName'][i]] = df['count'][i]

for i in range (df2.shape[0]):
    if df2['LastName'][i] not in d.keys():
        d[df2['LastName'][i]] = 0
    d[df2['LastName'][i]] += df2['count'][i]

In [ ]:
l=[]
for i in train['LastName']:
    if(d[i]>=5):
        l.append(i)
    else:
        l.append('RandomSurname')
    
train['LastName'] = l


l=[]
for i in test['LastName']:
    if(d[i]>=5):
        l.append(i)
    else:
        l.append('RandomSurname')
    
test['LastName'] = l

In [ ]:
## Encoding CATEGORICAL variables Sex, Embarked
le =LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'])
test['Sex'] = le.transform(test['Sex'])
train['Embarked'] = le.fit_transform(train['Embarked'])
test['Embarked'] = le.transform(test['Embarked'])
train['Ticket'] = le.fit_transform(train['Ticket'])
test['Ticket'] = le.transform(test['Ticket'])
le.fit(list(train['LastName'])+list(test['LastName']))
train['LastName']=le.transform(train['LastName'])
test['LastName'] = le.transform(test['LastName'])
train['Cabin']=le.fit_transform(train['Cabin'])
test['Cabin'] = le.transform(test['Cabin'])

In [ ]:
sns.countplot(x='Parch',hue='Survived',data=train)

In [ ]:
sns.countplot(x='SibSp',hue='Survived',data=train)

### We can make another feature corelating with both Parch and Sibsp

In [ ]:
train['FamOnBoard'] = train['Parch']+train['SibSp']
test['FamOnBoard'] = test['Parch']+test['SibSp']
l=[]
l2=[]
for i in train['FamOnBoard']:
    if i ==0:
        l.append(1)
    else:
        l.append(0)


train['Alone'] = l




l=[]
l2=[]
for i in test['FamOnBoard']:
    if i ==0:
        l.append(1)
    else:
        l.append(0)

test['Alone'] = l


In [ ]:
X = train.drop(['PassengerId','Survived','Name'],axis=1)
X_test = test.drop(['PassengerId','Name'],axis=1)   
y = train['Survived']

In [ ]:
X.head()

In [ ]:
sns.countplot(x='FamOnBoard',hue='Survived',data=train)

In [ ]:
sns.countplot(x='Alone',hue='Survived',data=train)

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(X.corr(),annot=True,vmin=-1,vmax=1,cmap='coolwarm')

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2,random_state=42)

# Model Selection using Pycaret 🥕  

## Comparing baselines

In [ ]:
from pycaret.classification import *
clf1 = setup(data=train.drop(['PassengerId','Name'],axis=1),target='Survived',
             categorical_features= ['Pclass','Sex','Ticket','Embarked','Cabin'],
             silent=True,train_size=0.8,
             normalize=True, 
             create_clusters=True,
             ignore_low_variance=True,
             polynomial_features=True, 
             trigonometry_features=True,
             feature_interaction=True,
             feature_ratio=True,
             feature_selection=True )

In [ ]:
models()

In [ ]:
compare_models()

## Creating and Tuning Top 3 models

In [ ]:
lgbm = create_model('lightgbm')

In [ ]:
tuned_lgbm = tune_model(lgbm)

In [ ]:
cb = create_model('catboost',fold=5)

In [ ]:
tuned_cb = tune_model(cb,fold=5)

In [ ]:
gbc = create_model('gbc',fold=5)


In [ ]:
tuned_gbc = tune_model(gbc,fold=5)

## Blending

In [ ]:
blender = blend_models(estimator_list = [tuned_lgbm,tuned_cb,tuned_gbc], method = 'soft')

In [ ]:
blender2 = blend_models(estimator_list = [lgbm,cb,gbc], method = 'soft')

## Stacking

In [ ]:
stacker = stack_models(estimator_list = [tuned_lgbm,tuned_cb,tuned_gbc], meta_model=tuned_lgbm)

In [ ]:
plot_model(lgbm)

In [ ]:
plot_model(tuned_lgbm)

In [ ]:
plot_model(cb)

In [ ]:
plot_model(tuned_cb)

In [ ]:
plot_model(gbc)

In [ ]:
plot_model(tuned_gbc)

In [ ]:
plot_model(blender)

In [ ]:
plot_model(stacker)

## Making Predictions

### Individual model predictions

In [ ]:
df = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predict_model(tuned_lgbm,data=X_test)['Label']})
df.to_csv('submit.csv',index=False)

In [ ]:
df2 = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predict_model(tuned_cb,data=X_test)['Label']})
df2.to_csv('submit2.csv',index=False)

In [ ]:
df3 = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predict_model(tuned_gbc,data=X_test)['Label']})
df3.to_csv('submit3.csv',index=False)

### Blender Model Predictions

In [ ]:
df4 = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predict_model(blender,data=X_test)['Label']})
df4.to_csv('submit4.csv',index=False)

### Stacker Model Predictions

In [ ]:
df5 = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predict_model(stacker,data=X_test)['Label']})
df5.to_csv('submit5.csv',index=False)

In [ ]:
df6 = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predict_model(lgbm,data=X_test)['Label']})
df6.to_csv('submit6.csv',index=False)